In [1]:
import pandas as pd
import numpy as np

In [72]:
def update(ex):
    
#     print(ex['id'].unique())
    
    ex = ex.drop_duplicates(subset = ['Days','PAY_DAY_SUPPLY_CNT_Opioid'], keep = 'last')
    ex = ex.reset_index()
    ex.drop(columns = ['index'], inplace = True)
    for i in range(0,ex.shape[0]):

        ex.loc[i,'last_day_opioid'] = int(ex.loc[i,'Days']) + int(ex.loc[i,'PAY_DAY_SUPPLY_CNT_Opioid']) - 1


    #check if the next script is within the range of dates of the previous script to change the last day opioid
    for i in range(1,ex.shape[0]):
        if ex.loc[i, 'Days'] in range(ex.loc[i-1,'Days'], ex.loc[i-1,'last_day_opioid'] + 1):
            ex.loc[i-1,'last_day_opioid'] = ex.loc[i,'Days']-1
        else:
            continue
            
     ################################################      
            
    ex[['PAY_DAY_SUPPLY_CNT_Opioid']] = ex[['PAY_DAY_SUPPLY_CNT_Opioid']].fillna(value = 0)
    check_days = [i for i in ex['Days'].tolist() if i <= 180]
  

    if len(check_days) > 0:
        days = np.array(check_days)
 
        lst = ex['PAY_DAY_SUPPLY_CNT_Opioid'].tolist()
        opioid_cnt = np.array(lst[:len(check_days)])
        length = (days + opioid_cnt).tolist()
        idx = [i for i in length if i > 180]
        lower_idx = [i for i in length if i <= 180]

        
        #last day opioid
        if len(idx) > 0:

            x = length.index(idx[0])
            ex = ex.iloc[:x+1,:]
            
            if ex.iloc[-1,-3] > 180:
                ex.iloc[-1,-3] = 180

            
        #changing the amount supply count till date 180th
            if (180 - ex.iloc[-1,2]) > ex.iloc[-1,3]:

                ex.iloc[-1,3] = 180 - ex.iloc[-1,2]

        else:
            ex = ex.iloc[:len(lower_idx),:]
            #days + supply count > 180 then the last day = 180 and the supply_count = 180 - days
            if (ex.iloc[-1,2] + ex.iloc[-1,3]) > 180:

                ex.iloc[-1,-3] = 180
                ex.iloc[-1,3] = 180-ex.iloc[-1,2]
            else:
                ex.iloc[-1,-3] = ex.iloc[-1,2] + ex.iloc[-1,3] - 1
                   
                    
    ################################################
    ex['naive_claim'] = 0
    for i in range(1,ex.shape[0]):
        if np.abs(ex.loc[i,'Days'] - ex.loc[i-1,'last_day_opioid']) > 90:
            ex.loc[i,'naive_claim'] = 1
        else:
            continue

    return ex

In [ ]:
x = x[['id','event_descr','Days','PAY_DAY_SUPPLY_CNT_Opioid','MME_Opioid']]
x = x[(~x['MME_Opioid'].isna()) & (x['event_descr']=='RX Claim - Paid')]
x = x.reset_index().drop(columns = ['index'])

#intitialize the columns 'last_day_opioid','LTOT_prior','LTOT_after' with 0 values
x['last_day_opioid'] = 0
x['LTOT_prior'] = 0
x['LTOT_after'] = 0

update(x)

In [76]:
def LTOT_id(ex):
    ''' This function will create the columns:
    - "last_day_opioid"
    - "LTOT prior" - LTOT prior to the qualifying event (negative dates)
    - "LTOT after" - LTOT after the qualifying event (positive dates)
    - Anything that begins with # is commented out and not part of the function
    '''     
        
                        ###############################################################
    # DATES PRIOR 0
    #please refer to the STARTING DATE 0 ONWARDS (below) for syntax and explanation

    neg_lst = [i for i in ex['Days'].tolist() if i < 0]
    idx_lst = [ex['Days'].tolist().index(i) for i in neg_lst]
    for i in range(0,len(idx_lst)):
        if ex.loc[i,'Days'] < 0:
            temp_lst = ex.loc[0:i,'Days'].tolist()
            date_range = ex.loc[i,'Days'] - 180
            #get the indices of the previous days if they are in the range of the current day - 180:
            qualified_prev = [t for t in temp_lst if t in range(date_range, ex.loc[i,'Days'] + 1)]
            qualified_idx = [ex['Days'].tolist().index(i) for i in qualified_prev]
            opioid_cnt = ex.loc[qualified_idx, 'PAY_DAY_SUPPLY_CNT_Opioid'].sum()
            if opioid_cnt > 162:
                ex.loc[i,'LTOT_prior'] = 1
            else:
                continue
        else:
            continue

            
            ###############################################################

    # #STARTING DATE 0 ONWARDS
    
    #get a list of positive days from the "Days" column
    pos_lst = [i for i in ex['Days'].tolist() if i >= 0]
    #get the indices of those dates in the pos_lst
    idx_pos = [ex['Days'].tolist().index(i) for i in pos_lst]

#     print(idx_pos)
    
    if len(idx_pos) > 0:
        #iterate over each element in the idx_pos list
        for i in range(idx_pos[0],idx_pos[-1] + 1):

            #create a list called 'temp_lst' which select all the days in Days list starting from the first index to the i-th index (i-th index is the current element we're looking at)
            temp_lst = ex.loc[idx_pos[0]:i,'Days'].tolist()
            #we calculate the date range date (looking backwards 180 days before) between (the current date - 180) and the current day
            date_range = ex.loc[i,'last_day_opioid'] - 180
    #         print(date_range)

            #check if the 180th date before is not negative
            if date_range >= 0:
                #get the indices of the previous days if they are in the range of (the current day - 180) and current day - NOTICE: range(x,y) indicates range of time inclusive from x but exclusive of y - therefore, we must add 1:
                
                qualified_prev = [t for t in temp_lst if t in range(date_range,int(ex.loc[i,'last_day_opioid']) + 1)]
                
    #             print(qualified_prev)
                #get the indices of the dates which are in the range of (the current day - 180) and the current day
                qualified_idx = [ex['Days'].tolist().index(i) for i in qualified_prev]
    #                 print(qualified_idx)
                #get the sum of all the SUPPLY_DAY_CNT from those rows that are in the qualified_idx - Sum of on-hand opioid in the previous 180 days
                opioid_cnt = ex.loc[qualified_idx, 'PAY_DAY_SUPPLY_CNT_Opioid'].sum()
                #if the opioid count > 162 within the 180 days - note it to be 1 else 0
                if opioid_cnt > 162:
                    ex.loc[i,'LTOT_after'] = 1
                else:
                    continue

            else:
                
                qualified_prev = [t for t in temp_lst if t in range(0, int(ex.loc[i,'last_day_opioid']) + 1)]
                qualified_idx = [ex['Days'].tolist().index(i) for i in qualified_prev]
    #             print(qualified_idx)
                opioid_cnt = ex.loc[qualified_idx, 'PAY_DAY_SUPPLY_CNT_Opioid'].sum()
                if opioid_cnt > 162:
                    ex.loc[i,'LTOT_after'] = 1
                else:
                    continue
 
            
    return ex

In [77]:
df = pd.read_csv('humana_3.csv')
df.drop(columns = ["Unnamed: 0","Unnamed: 0.1"], inplace = True)

/Users/duytienpham/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
#select only these relevant columns for this calculation
df2 = df[['id','event_descr','Days','PAY_DAY_SUPPLY_CNT_Opioid','MME_Opioid']]
df2 = df2[(~df2['MME_Opioid'].isna()) & (df2['event_descr']=='RX Claim - Paid')]
df2 = df2.reset_index().drop(columns = ['index'])

#intitialize the columns 'last_day_opioid','LTOT_prior','LTOT_after' with 0 values
df2['last_day_opioid'] = 0
df2['LTOT_prior'] = 0
df2['LTOT_after'] = 0

memID = df2['id'].unique().tolist()
table = pd.DataFrame()
for i in memID:
    ex = df2[df2['id']==i]
    ex = ex.reset_index().drop(columns ='index')
    ex = update(ex)
    ex = LTOT_id(ex)
    table = pd.concat([table,ex])
    
    



In [79]:
table.to_csv('LTOT_identification.csv')

In [89]:
x = df[(df['id'] == 'ID10010854159') & (~df['MME_Opioid'].isna())]
x = x[['id','event_descr','Days','PAY_DAY_SUPPLY_CNT_Opioid','MME_Opioid']]